In [1]:
import os

# 获取当前工作目录
current_dir = os.getcwd()
print("当前工作目录：", current_dir)
 
# 切换到上一层目录
parent_dir = os.path.dirname(current_dir)
os.chdir(parent_dir)
print("切换后的目录：", parent_dir)

当前工作目录： c:\Users\zhaoxs3\Documents\GitHub\NEMESIS\unit_test
切换后的目录： c:\Users\zhaoxs3\Documents\GitHub\NEMESIS


In [2]:
import QuantLib as ql
import numpy as np
import pandas as pd

from devlib.market.curves.overnight_index_curves import Sofr, Estr
from devlib.market.curves.credit_curve_generator import MarketDataCreditCurve, CdsConfig
from devlib.products.credit.cds import Cds

from devlib.utils.ql_date_utils import ql_date_str
from devlib.utils.cds_utils import cds_maturity_date, calendar_5u

In [3]:
from nemesis.products.credit import *
from nemesis.utils import *

####################################################################
#  NEMESIS ALPHA Version 0.1.0 - This build: 24 Jan 2025 at 10:42 #
####################################################################



### Test Case 1

In [4]:
today = ql.Date(1,11,2024)
ql.Settings.instance().evaluationDate = today

mkt_file_path = './unit_test/data/ICVS530_curve_data_20241101.xlsx'
swap_mkt_data = pd.read_excel(mkt_file_path, sheet_name='swap')
discount_curve = Estr(today, swap_mkt_data=swap_mkt_data)

entity = 'EUROPE'
mkt_file_path = './unit_test/data/credit_curve_data_20241101.xlsx'
cds_type = 'index'
cds_data = pd.read_excel(mkt_file_path, sheet_name=cds_type)

cds_config = CdsConfig(recovery_rate = 0.4,
                        daycount = ql.Actual360(),
                        calendar = ql.WeekendsOnly())
daycount = ql.Actual360()

credit_curve = MarketDataCreditCurve(today, entity, cds_data, discount_curve, cds_type, cds_config, daycount)

In [5]:
value_dt = Date(1,11,2024)
issuer_curve = QLCreditCurve(value_dt, credit_curve)

In [6]:
cds_contract = GeneralCDS(
    step_in_dt=value_dt,
    maturity_dt_or_tenor="10Y",
    running_cpn=0.01,
    notional=1e7,
    long_protect=True,
    freq_type=FrequencyTypes.QUARTERLY,
    dc_type=DayCountTypes.ACT_360,
    cal_type=CalendarTypes.WEEKEND,
    bd_type=BusDayAdjustTypes.FOLLOWING,
    dg_type=DateGenRuleTypes.BACKWARD
)

In [7]:
cds_contract.value(value_dt, issuer_curve)

{'dirty_pv': np.float64(-29153.04997832945),
 'clean_pv': np.float64(-17208.60553388507)}

In [8]:
cds_contract.credit_dv01(value_dt, issuer_curve, 0.4, bump=1)

np.float64(8637.132547485875)

In [9]:
cds_contract.interest_dv01(value_dt, issuer_curve, 0.4)

np.float64(8.566898120916449)

### Test Case 2

In [10]:
today = ql.Date(1,11,2024)
ql.Settings.instance().evaluationDate = today

mkt_file_path = './unit_test/data/ICVS531_curve_data_20241101.xlsx'
swap_mkt_data = pd.read_excel(mkt_file_path, sheet_name='swap')
discount_curve = Sofr(today, swap_mkt_data=swap_mkt_data)

entity = 'CHINAGOV'
mkt_file_path = './unit_test/data/credit_curve_data_20241101.xlsx'
cds_type = 'single_name'
cds_data = pd.read_excel(mkt_file_path, sheet_name=cds_type)

cds_config = CdsConfig(recovery_rate = 0.4,
                        daycount = ql.Actual360(),
                        calendar = calendar_5u())
daycount = ql.Actual360()

credit_curve = MarketDataCreditCurve(today, entity, cds_data, discount_curve, cds_type, cds_config, daycount)

In [11]:
value_dt = Date(1,11,2024)
issuer_curve = QLCreditCurve(value_dt, credit_curve)

In [12]:
cds_contract = GeneralCDS(
    step_in_dt=value_dt,
    maturity_dt_or_tenor="10Y",
    running_cpn=0.05,
    notional=1e7,
    long_protect=True,
    freq_type=FrequencyTypes.QUARTERLY,
    dc_type=DayCountTypes.ACT_360,
    cal_type=CalendarTypes.CDS,
    bd_type=BusDayAdjustTypes.FOLLOWING,
    dg_type=DateGenRuleTypes.BACKWARD
)

In [13]:
cds_contract.value(value_dt, issuer_curve)

{'dirty_pv': np.float64(-3281982.6643738346),
 'clean_pv': np.float64(-3222260.4421516117)}

In [14]:
cds_contract.credit_dv01(value_dt, issuer_curve, 0.4, bump=1)

np.float64(10517.781088938937)

In [15]:
cds_contract.interest_dv01(value_dt, issuer_curve, 0.4)

np.float64(1502.1983877439052)

### Test Case 3

In [16]:
today = ql.Date(1,11,2024)
ql.Settings.instance().evaluationDate = today

mkt_file_path = './unit_test/data/ICVS531_curve_data_20241101.xlsx'
swap_mkt_data = pd.read_excel(mkt_file_path, sheet_name='swap')
discount_curve = Sofr(today, swap_mkt_data=swap_mkt_data)

entity = 'CHINAGOV'
mkt_file_path = './unit_test/data/credit_curve_data_20241101.xlsx'
cds_type = 'single_name'
cds_data = pd.read_excel(mkt_file_path, sheet_name=cds_type)

cds_config = CdsConfig(recovery_rate = 0.4,
                        daycount = ql.Actual360(),
                        calendar = calendar_5u())
daycount = ql.Actual360()

credit_curve = MarketDataCreditCurve(today, entity, cds_data, discount_curve, cds_type, cds_config, daycount)

In [17]:
value_dt = Date(1,11,2024)
issuer_curve = QLCreditCurve(value_dt, credit_curve)

In [18]:
cds_contract = GeneralCDS(
    step_in_dt=value_dt,
    maturity_dt_or_tenor=Date(1,11,2029),
    running_cpn=0.0,
    upfront_payment_dt=Date(6,11,2024),
    upfront_amount=3e5,    
    notional=1e7,
    long_protect=True,
    freq_type=FrequencyTypes.QUARTERLY,
    dc_type=DayCountTypes.ACT_360,
    cal_type=CalendarTypes.WEEKEND,
    bd_type=BusDayAdjustTypes.FOLLOWING,
    dg_type=DateGenRuleTypes.BACKWARD
)

In [19]:
cds_contract.value(value_dt, issuer_curve)

np.float64(-6439.825080876122)

In [20]:
cds_contract.credit_dv01(value_dt, issuer_curve, 0.4, bump=1)

np.float64(4377.223720230453)

In [21]:
cds_contract.interest_dv01(value_dt, issuer_curve, 0.4)

np.float64(-71.9812481815461)

### Test Case 4

In [22]:
today = ql.Date(1,11,2024)
ql.Settings.instance().evaluationDate = today

mkt_file_path = './unit_test/data/ICVS531_curve_data_20241101.xlsx'
swap_mkt_data = pd.read_excel(mkt_file_path, sheet_name='swap')
discount_curve = Sofr(today, swap_mkt_data=swap_mkt_data)

entity = 'CHINAGOV'
mkt_file_path = './unit_test/data/credit_curve_data_20241101.xlsx'
cds_type = 'single_name'
cds_data = pd.read_excel(mkt_file_path, sheet_name=cds_type)

cds_config = CdsConfig(recovery_rate = 0.4,
                        daycount = ql.Actual360(),
                        calendar = calendar_5u())
daycount = ql.Actual360()

credit_curve = MarketDataCreditCurve(today, entity, cds_data, discount_curve, cds_type, cds_config, daycount)

In [23]:
value_dt = Date(1,11,2024)
issuer_curve = QLCreditCurve(value_dt, credit_curve)

In [24]:
cds_contract = GeneralCDS(
    step_in_dt=value_dt,
    maturity_dt_or_tenor=Date(1,11,2029),
    running_cpn=0.05,
    upfront_payment_dt=Date(6,11,2024),
    upfront_amount=0,    
    notional=1e7,
    coupon_pay_front=True,
    long_protect=True,
    freq_type=FrequencyTypes.QUARTERLY,
    dc_type=DayCountTypes.ACT_360,
    cal_type=CalendarTypes.WEEKEND,
    bd_type=BusDayAdjustTypes.FOLLOWING,
    dg_type=DateGenRuleTypes.BACKWARD,
    ac_type=AccCouponTypes.TO_PERIOD_END,
    acp_type=AccCouponPayTypes.DEFAULT
)

In [25]:
cds_contract.value(value_dt, issuer_curve)

{'dirty_pv': np.float64(-1849684.0720076142),
 'clean_pv': np.float64(-1849684.0720076142)}

In [26]:
cds_contract.credit_dv01(value_dt, issuer_curve, 0.4, bump=1)

np.float64(5250.860551352147)

In [27]:
cds_contract.interest_dv01(value_dt, issuer_curve, 0.4)

np.float64(433.4254135877127)

### Test Case 5

In [28]:
today = ql.Date(1,11,2024)
ql.Settings.instance().evaluationDate = today

mkt_file_path = './unit_test/data/ICVS531_curve_data_20241101.xlsx'
swap_mkt_data = pd.read_excel(mkt_file_path, sheet_name='swap')
discount_curve = Sofr(today, swap_mkt_data=swap_mkt_data)

entity = 'CHINAGOV'
mkt_file_path = './unit_test/data/credit_curve_data_20241101.xlsx'
cds_type = 'single_name'
cds_data = pd.read_excel(mkt_file_path, sheet_name=cds_type)

cds_config = CdsConfig(recovery_rate = 0.4,
                        daycount = ql.Actual360(),
                        calendar = calendar_5u())
daycount = ql.Actual360()

credit_curve = MarketDataCreditCurve(today, entity, cds_data, discount_curve, cds_type, cds_config, daycount)

In [29]:
value_dt = Date(1,11,2024)
issuer_curve = QLCreditCurve(value_dt, credit_curve)

In [30]:
cds_contract = GeneralCDS(
    step_in_dt=value_dt,
    maturity_dt_or_tenor=Date(1,11,2029),
    running_cpn=0.05,
    upfront_payment_dt=Date(6,11,2024),
    upfront_amount=0,    
    notional=1e7,
    coupon_pay_front=True,
    long_protect=True,
    freq_type=FrequencyTypes.QUARTERLY,
    dc_type=DayCountTypes.ACT_360,
    cal_type=CalendarTypes.WEEKEND,
    bd_type=BusDayAdjustTypes.FOLLOWING,
    dg_type=DateGenRuleTypes.BACKWARD,
    ac_type=AccCouponTypes.TO_DEFAULT,
    acp_type=AccCouponPayTypes.DEFAULT
)

In [31]:
cds_contract.value(value_dt, issuer_curve)

{'dirty_pv': np.float64(-1846590.3833061182),
 'clean_pv': np.float64(-1846590.3833061182)}

In [32]:
cds_contract.credit_dv01(value_dt, issuer_curve, 0.4, bump=1)

np.float64(5297.757821913809)

In [33]:
cds_contract.interest_dv01(value_dt, issuer_curve, 0.4)

np.float64(432.6685442774324)